In [ ]:
import pyautogui
import time
import win32gui
import threading

In [ ]:
useOverload = False

In [ ]:
def windowEnumerationHandler(hwnd, top_windows):
    top_windows.append((hwnd, win32gui.GetWindowText(hwnd)))
    
def ColorIsAround(r,g,b,r2,g2,b2,t):
        return ((r<r2+t and r>r2-t) and (g<g2+t and g>g2-t) and (b<b2+t and b>b2-t))

In [ ]:
pyautogui.FAILSAFE = False
class Screen():
    def __init__(self,startx,starty,ApplicationNum):
        self.startx = startx
        self.starty = starty
        self.firstinvent = (startx + 570, starty + 282)
        self.appNum = ApplicationNum
        self.InventLocations = []
        self.playerHealth = 100
        self.currentPray = "Green"
        self.foodleft = 24
        self.done = False
        self.needReset = False
        self.killcount = 0
        self.currentOverloadDose = 6
        self.OverLoadTime = time.time()
        self.SumTime = time.time()
        self.timeSinceHit = time.time()
        self.checkPoint = 0
        self.Tx = 0
        self.Ty = 0
        for y in range(7):
            for x in range(4):
                self.InventLocations.append((self.firstinvent[0]+44*x, self.firstinvent[1]+37*y))
    def Eat(self):
        self.timeSinceHit = time.time()
        SpottoEat = 28 - (self.foodleft+4)
        self.popUp()
        pyautogui.press('f1')
        pyautogui.moveTo(self.InventLocations[SpottoEat])
        time.sleep(.45)
        self.foodleft -= 1
        if self.foodleft < 0:
            self.needReset = True
        pyautogui.click()
        time.sleep(.75)
        
    def PickUpItems(self):
        Tilex,Tiley = self.FindMarkedTile()
        pyautogui.moveTo(Tilex,Tiley)
        moved = True
        time.sleep(.4)
        pyautogui.click()
        time.sleep(2)
        
        
        Tx, Ty = self.FindMarkedTile()
        pyautogui.moveTo(Tx,Ty)
        for x in range(10):
            pyautogui.scroll(1000)
        Tx, Ty = self.FindMarkedTile()
        pyautogui.moveTo(Tx,Ty)
        
        pyautogui.click()
        time.sleep(1)
        pyautogui.click()
        time.sleep(1)
        pyautogui.click()
        time.sleep(1)
        pyautogui.click()
        time.sleep(1)
        pyautogui.click()
        time.sleep(1)
        pyautogui.click()
        time.sleep(1)
        
        for x in range(10):
            pyautogui.scroll(-1000)
        
    def startKill(self):
        pyautogui.moveTo(int(545+self.startx) ,int(80+self.starty))
        time.sleep(.4)
        pyautogui.click()
        
        self.foodleft = 24
        pyautogui.moveTo(250, 250)
        pyautogui.mouseDown(button='middle')
        pyautogui.moveTo(250, 400,.1)
        pyautogui.mouseUp(button='middle')
        
        for x in range(5):
            pyautogui.scroll(-1000)
        time.sleep(1)
        self.Pray("Green")
        self.currentPray = "Green"
        self.popUp()
        pyautogui.press('f4')
        pyautogui.moveTo(int(575+self.startx),int(445+self.starty))
        pyautogui.click(button='right')
        pyautogui.click(int(575+self.startx) ,int(488+self.starty))
        self.timeSinceHit = time.time()
        
    def UseSummoning(self):
        self.popUp()
        pyautogui.press('f5')
        pyautogui.moveTo(int(580+self.startx) ,int(313+self.starty))
        time.sleep(.3)
        pyautogui.click()
        self.popUp()
        im = pyautogui.screenshot()
        if ColorIsAround(im.getpixel((640,358))[0],im.getpixel((640,358))[1],im.getpixel((640,358))[2],31,28,23,8):
            self.needReset = True
        
    def Pray(self,Type):
        self.popUp()
        pyautogui.press('f3')
        if Type == "Red":
            #print('Pray Red')
            pyautogui.moveTo(int(715+self.startx) ,int(390+self.starty))
        if Type == "Blue":
            #print('Pray Blue')
            pyautogui.moveTo(int(640+self.startx),int(390+self.starty))
        if Type == "Green":
            #print('Pray Green')
            pyautogui.moveTo(int(680+self.startx),int(390+self.starty))
        time.sleep(.8)
        pyautogui.click()
        time.sleep(.7)
    def popUp(self):
        win32gui.ShowWindow(self.appNum,5)
        win32gui.SetForegroundWindow(self.appNum)
        
    def CheckHealth(self):
        self.popUp()
        im = pyautogui.screenshot()
        if(im.getpixel((int(704+self.startx),int(77+self.starty)))[1]>45):
            self.playerHealth = 80
        else:
            self.playerHealth = 100
        if(im.getpixel((int(709+self.startx), int(82+self.starty)))[1]>45):
            self.playerHealth = 70
        if(im.getpixel((int(709+self.startx), int(85+self.starty)))[1]>45):
            self.playerHealth = 60
    
    def FindMarkedTile(self):
        endx = 515 + self.startx
        endy = 400 + self.starty
        startx =  self.startx-5
        starty =  self.starty+55
        self.popUp()
        im = pyautogui.screenshot()
        places = []
        for y in range (starty,endy):
            for x in range(startx,endx):
                if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],211,211,0,15):
                    places.append((x,y))
        
        if len(places) <= 1:
            return 0,0 
        x = int((places[0][0]+ places[len(places)-1][0])/2)
        y = int((places[0][1]+ places[len(places)-1][1])/2)
        return x,y
    def ZulrahSearch(self):
        endx = 515 + self.startx
        endy = 400 + self.starty
        startx =  self.startx-5
        starty =  self.starty+55
        self.popUp()
        im = pyautogui.screenshot()
        for y in range (starty,endy,2):
            for x in range(startx,endx,2):
                if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],184,205,24,10):
                    #print("Green")
                    return(x, y, "Green")
                if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],244,200,62,10):
                    #print("Red")
                    return(x, y, "Red")
                if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],38,251,200,10):
                    #print("Blue")
                    return(x, y, "Blue")
        return(0,0,"NA")
    def FindandClickHeal(self):
        pyautogui.moveTo(95+self.startx,95+self.starty)
        for x in range(2):
            pyautogui.scroll(500)
        
        
        endx = 515 + self.startx
        endy = 400 + self.starty
        startx =  self.startx-5
        starty =  self.starty+55
        self.popUp()
        im = pyautogui.screenshot()
        for y in range (starty,endy,2):
            for x in range(startx,endx,2):
                if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],157,235,246,5):
                    pyautogui.moveTo(x,y)
                    time.sleep(.3)
                    pyautogui.click()
                    for x in range(4):
                        pyautogui.scroll(-500)
                    return
    def Bank(self):
        if((time.time() - self.OverLoadTime) >  310):
            self.OverLoad()
            self.OverLoadTime = time.time()
            time.sleep(1)
        
        
        self.done = False
        self.needReset = False
        pyautogui.moveTo(int(545+self.startx) ,int(80+self.starty))
        time.sleep(.4)
        pyautogui.click()
        time.sleep(.4)
        pyautogui.moveTo(int(100+self.startx) ,int(80+self.starty))
        pyautogui.scroll(-6000)
        
        
        pyautogui.press('f4')
        pyautogui.moveTo(self.InventLocations[0])
        time.sleep(.3)
        pyautogui.click()
        time.sleep(7.5)
        
        pyautogui.press('f2')
        pyautogui.moveTo(int(692+self.startx),int(437+self.starty))
        time.sleep(.3)
        pyautogui.click(button='right')
        pyautogui.moveTo(int(692+self.startx),int(437+self.starty+42))
        time.sleep(.4)
        pyautogui.click()
        
        time.sleep(1)
        pyautogui.press('space')
        
        time.sleep(1.4)
        pyautogui.press('1')
        time.sleep(1)
        
        
        
        self.FindandClickHeal()
        time.sleep(3)
        
        pyautogui.moveTo(int(632 + self.startx),int(187 + self.starty))
        time.sleep(.2)
        pyautogui.click()           
        time.sleep(6)
        x = 0
        y = 0
        while x == 0 and y == 0:
            x,y = self.FindMarkedTile()
            if x ==0 and y ==0:
                return self.Bank()
        pyautogui.moveTo(int(x),int(y))
        time.sleep(.6)
        pyautogui.click()
        time.sleep(5.5)
        
        #im = pyautogui.screenshot()
        #while (ColorIsAround(im.getpixel((200,100))[0],im.getpixel((200,100))[1],im.getpixel((200,100))[2],72,62,51,8) == False):
        #    time.sleep(1)
        #    im = pyautogui.screenshot()
       # pyautogui.moveTo(int(285+self.startx),int(223+self.starty))
        #time.sleep(.4)
        #pyautogui.click()            #IntoBank
        #time.sleep(1.5)
        
        pyautogui.moveTo(int(342+self.startx),int(355+self.starty))
        time.sleep(.6)
        pyautogui.click()         #Into Pre Sets
        time.sleep(3)
        
        if(self.currentOverloadDose == 6):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty))
        elif(self.currentOverloadDose == 5):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty +(22*1)))
        elif(self.currentOverloadDose == 4):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty +(22*2)))
        elif(self.currentOverloadDose == 3):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty +(22*3)))
        elif(self.currentOverloadDose == 2):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty +(22*4)))
        elif(self.currentOverloadDose == 1):
            pyautogui.moveTo(int(170+self.startx),int(172+self.starty +(22*5)))
         
        time.sleep(.6)
        pyautogui.click()          #Into ZulrahPreSet
        time.sleep(3.5)
        
        pyautogui.moveTo(int(407+self.startx),int(325+self.starty))
        time.sleep(.2)
        pyautogui.click()            # Accepting Preset
        time.sleep(1.5)
        
    def action(self):
        self.popUp()
        if self.checkPoint == 0: # begins kill
            self.startKill()
            time.sleep(6)
            self.checkPoint += 1
            return
        if self.checkPoint == 1: # during kill
            self.Update()
            if self.done == True or self.needReset == True:
                self.checkPoint += 1
            return
        if self.checkPoint == 2: # picking up items (walking to center-tile)
            Tilex,Tiley = self.FindMarkedTile()
            pyautogui.moveTo(Tilex,Tiley)
            moved = True
            time.sleep(.4)
            pyautogui.click()
            self.tempTime = time.time()
            self.checkPoint += 1
            return
        if self.checkPoint == 3: # zoom into center-tile
            if time.time() - self.tempTime > 2:
                Tx, Ty = self.FindMarkedTile()
                pyautogui.moveTo(Tx,Ty)
                for x in range(10):
                    pyautogui.scroll(1000)
                Tx, Ty = self.FindMarkedTile()
                self.Tx = Tx
                self.Ty = Ty
                pyautogui.moveTo(Tx,Ty)
                self.checkPoint += 1
            return
        if self.checkPoint == 4:
            pyautogui.moveTo(self.Tx,self.Ty)
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            self.checkPoint += 1
            return
        if self.checkPoint == 5:
            pyautogui.moveTo(self.Tx,self.Ty)
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            pyautogui.click()
            time.sleep(.5)
            self.checkPoint += 1
            return
        if self.checkPoint == 6:
            self.done = False
            self.needReset = False
            pyautogui.moveTo(int(545+self.startx) ,int(80+self.starty))
            time.sleep(.4)
            pyautogui.click()
            time.sleep(.4)
            pyautogui.moveTo(int(100+self.startx) ,int(80+self.starty))
            pyautogui.scroll(-6000)
            time.sleep(.25)
            self.checkPoint += 1
            return
        if self.checkPoint == 7:
            pyautogui.press('f4')
            pyautogui.moveTo(self.InventLocations[0])
            time.sleep(.3)
            pyautogui.click()
            self.tempTime = time.time()
            self.checkPoint += 1
            return
        if self.checkPoint == 8:
            if time.time() - self.tempTime > 7.5:
                pyautogui.press('f2')
                pyautogui.moveTo(int(692+self.startx),int(437+self.starty))
                time.sleep(.3)
                pyautogui.click(button='right')
                pyautogui.moveTo(int(692+self.startx),int(437+self.starty+42))
                time.sleep(.4)
                pyautogui.click()
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 9:
            if time.time() - self.tempTime > 1:
                pyautogui.press('space')
                self.checkPoint += 1
                self.tempTime = time.time()
                time.sleep(.25)
            return
        if self.checkPoint == 10:
            if time.time() - self.tempTime > 1.5:
                pyautogui.press('1')
                self.tempTime = time.time()
                time.sleep(.25)
                self.checkPoint += 1
            return
        if self.checkPoint == 11:
            if time.time() - self.tempTime > 1:
                self.FindandClickHeal()
                time.sleep(1.5)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 12:
            if time.time() - self.tempTime > 3:
                pyautogui.moveTo(int(632 + self.startx),int(187 + self.starty))
                time.sleep(.2)
                pyautogui.click()
                time.sleep(.3)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 13:
            if time.time() - self.tempTime > 6:
                x = 0
                y = 0
                x,y = self.FindMarkedTile()
                if x == 0 and y == 0:
                    self.checkPoint = 6
                    return
                pyautogui.moveTo(int(x),int(y))
                time.sleep(.6) # modifiable
                pyautogui.click()
                time.sleep(.3)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 14:
            if time.time() - self.tempTime > 5.5:
                pyautogui.moveTo(int(342+self.startx),int(355+self.starty))
                time.sleep(.6)
                pyautogui.click()
                time.sleep(.3)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 15:
            if time.time() - self.tempTime > 3:
                pyautogui.moveTo(int(170+self.startx),int(172+self.starty))    
                time.sleep(.6)
                pyautogui.click()
                time.sleep(.3)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 16:
            if time.time() - self.tempTime > 3.5:
                pyautogui.moveTo(int(407+self.startx),int(325+self.starty))
                time.sleep(.2)
                pyautogui.click() 
                time.sleep(1.5)
                self.checkPoint += 1
                self.tempTime = time.time()
            return
        if self.checkPoint == 17:
            if time.time() - self.tempTime > 1.5:
                self.checkPoint = 0
            return
        
    def Update(self):
        self.popUp()
        now = time.time()
        self.CheckHealth()
        if(self.playerHealth < 100):
            self.Eat()
        if(self.needReset == True):
                self.done = True
                return
        im = pyautogui.screenshot()
        if(ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],70,170,8,5) or ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],187,28,23,5)):
            self.timeSinceHit = time.time()
        if((time.time()-self.SumTime) > 11 and ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],70,170,8,5)):
                #print("summoning")
                self.UseSummoning()
                self.SumTime = time.time()
        pray = "NA"
        moved = False
        im = pyautogui.screenshot()
        if(ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],70,170,8,5) == False and ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],187,28,23,5) == False):
            if((time.time() - self.timeSinceHit) > 7 and (time.time() - self.timeSinceHit) < 8):
                moved = False
            if((time.time() - self.timeSinceHit) > 15.5):
                print(time.time() - self.timeSinceHit)
                self.done = True
                self.Pray(self.currentPray)
                self.killcount+=1
                #print("I have killed this many Zulrahs", self.killcount)
                return
            x,y,pray = self.ZulrahSearch()
            if pray != self.currentPray and pray != "NA":
                #print(time.time()-self.timeSinceHit)
                #print(time.time()-now)
                self.Pray(pray)
                self.currentPray = pray
            if x != 0 and y != 0:
                pyautogui.moveTo(x,y)
                if((time.time()-self.SumTime) > 11 and ColorIsAround(im.getpixel((40,110))[0],im.getpixel((40,110))[1],im.getpixel((40,110))[2],70,170,8,5)):
                    #print("summoning")
                    self.UseSummoning()
                    self.SumTime = time.time()
                time.sleep(.4)
                pyautogui.click()
                time.sleep(.4)
                pyautogui.moveTo(int(545+self.startx) ,int(80+self.starty))
                time.sleep(.4)
                pyautogui.click()
                self.timeSinceHit = time.time()
                return
            else:
                if moved == False:
                    Tilex,Tiley = self.FindMarkedTile()
                    if Tilex == 0 and Tiley == 0:
                        pyautogui.moveTo(250, 250)
                        pyautogui.mouseDown(button='middle')
                        pyautogui.moveTo(550, 250,.1)
                        pyautogui.mouseUp(button='middle')
                        Tilex,Tiley = self.FindMarkedTile()
                    pyautogui.moveTo(Tilex,Tiley)
                    moved = True
                    time.sleep(.6)
                    pyautogui.click()
                    time.sleep(1)
                    #print("move")
                    
    def OverLoad(self):
        if useOverload == True:
            self.popUp()
            pyautogui.press('f1')
            pyautogui.moveTo(self.InventLocations[24])
            time.sleep(.4)
            pyautogui.click()
            time.sleep(.4)
            if self.currentOverloadDose != 1:
                self.currentOverloadDose -=1
            else:
                self.currentOverloadDose = 6
class BotFarm():
    def __init__(self):
        self.Screens = []
        self.NumOfScreen = 0
        results = []
        top_windows = []
        win32gui.EnumWindows(windowEnumerationHandler, top_windows)
        for pos in pyautogui.locateAllOnScreen("C:/Users/User/Downloads/Ikov.png"):
            self.startx = pos[0]
            self.starty = pos[1]
        for i in top_windows:
            if "ikov" in i[1].lower():
                print(i)
                win32gui.ShowWindow(i[0],5)
                win32gui.SetForegroundWindow(i[0])
                self.NumOfScreen += 1
                self.Screens.append(Screen(pos[0],pos[1],i[0]))
                
    def startBot(self,num):
        for Screen in self.Screens:
            Screen.popUp()
            Screen.StartKill()
            self.Screens[num].popUp()
            self.Screens[num].OverLoad()
            while True:
                self.Screens[num].popUp()
                self.Screens[num].StartKill()
                now = time.time()
                while self.Screens[num].done == False:
                    self.Screens[num].Update()
                if(self.Screens[num].needReset == False):
                    self.Screens[num].PickUpItems()
                self.Screens[num].Bank()
            

In [ ]:
time.sleep(4)
for x in range(3):
            pyautogui.scroll(500)

In [ ]:
New = BotFarm()

In [ ]:
while True:
    New.Screens[0].action()
    time.sleep(1/4)
    #New.Screens[1].action()
    #time.sleep(1/4)

In [ ]:
t1 = threading.Thread(target=New.startBot, args=(0,)) 
t2 = threading.Thread(target=New.startBot, args=(1,))
try:
    t1.start()
    t2.start()
    t1.join()
    t2.join()
except KeyboardInterrupt:
    print('\n')


In [ ]:
New.Screens[0].PopUp()
New.Screens[0].OverLoad()
while True:
    New.Screens[0].PopUp()
    New.Screens[0].StartKill()
    now = time.time()
    while New.Screens[0].done == False:
        New.Screens[0].Update()
    if(New.Screens[0].needReset == False):
        New.Screens[0].PickUpItems()
    New.Screens[0].Bank()
    

In [ ]:
New.Screens[0].PopUp()

In [ ]:
for x in range(4):
    print(New.Screens[x].startx , New.Screens[x].starty)

In [ ]:
New.Screens[0].PopUp()
New.Screens[0].Bank()

In [ ]:
New.Screens[0].PopUp()
New.Screens[0].PickUpItems()

In [ ]:
locations = New.Screens[0].FindMarkedTile()

In [ ]:
locations

In [ ]:

time.sleep(5)
pyautogui.moveTo(10,10)
pyautogui.click()
time.sleep(.5)
pyautogui.click()

In [ ]:
(416+464)/2

In [ ]:
(250+348)/2

In [ ]:
def ScrollScreen(x,y):
    pyautogui.moveTo(250, 250)
    pyautogui.mouseDown(button='middle')
    pyautogui.moveTo(250, 400,.1)
    pyautogui.mouseUp(button='middle')

In [ ]:
def Click():
    pyautogui.moveTo(10,10)
    pyautogui.click()
    time.sleep(.5)
    pyautogui.click()

In [ ]:
ActionsQ = []

In [ ]:
ActionsQ.append(ScrollScreen(100,100))

In [ ]:
ActionsQ

In [ ]:
def ColorIsAround(r,g,b,r2,g2,b2,t):
    return ((r<r2+t and r>r2-t) and (g<g2+t and g>g2-t) and (b<b2+t and b>b2-t))
def ZulrahSearch(Startx,Starty):
    now = time.time()
    endx = 515
    endy = 400
    startx =  Startx-5
    starty =  Starty+55
    im = pyautogui.screenshot()
    for y in range (starty,endy,6):
        for x in range(startx,endx,6):
            if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],184,205,24,10):
                print("Green")
                pyautogui.moveTo(x,y)
                return time.time()-now
            if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],244,200,62,10):
                print("Red")
                pyautogui.moveTo(x,y)
                return time.time()-now
            if ColorIsAround(im.getpixel((x,y))[0],im.getpixel((x,y))[1],im.getpixel((x,y))[2],38,251,200,10):
                print("Blue")
                pyautogui.moveTo(x,y)
                return time.time()-now

In [ ]:
if __name__ == "__main__":
    results = []
    top_windows = []
    win32gui.EnumWindows(windowEnumerationHandler, top_windows)
    for i in top_windows:
        if "ikov" in i[1].lower():
            print(i)
            win32gui.ShowWindow(i[0],5)
            win32gui.SetForegroundWindow(i[0])

In [ ]:
win32gui.SetForegroundWindow(132640)

In [ ]:
for i in top_windows:
            if "ikov" in i[1].lower():
                print(i)
                win32gui.ShowWindow(i[0],5)
                win32gui.SetForegroundWindow(i[0])
                
                for pos in pyautogui.locateAllOnScreen('C:\\Users\\colem\\Desktop\\Pictures\\Ikov.png'):
                    l = pos[0]
                    u = pos[1]
                    print(l,u)